In [1]:
import faiss

In [2]:
import numpy as np
data_distil_bert =  np.load("features_distilbert.npz")
final_data_distil_bert = []
for each in data_distil_bert:
    for each_one in data_distil_bert[each]:
        final_data_distil_bert.append(each_one)

final_data_distil_bert = np.array(final_data_distil_bert)

In [3]:
d = final_data_distil_bert.shape[1]
nlist = 14
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [4]:
index.is_trained

False

In [5]:
index.train(final_data_distil_bert)
index.is_trained  # check if index is now trained

True

In [6]:
index.add(final_data_distil_bert)

In [7]:
index.ntotal

728

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

: 

In [ ]:
def distil_bert_model():
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    return model,tokenizer

model, tokenizer = distil_bert_model()




In [87]:
def test_samples(model, tokenizer, text, max_len):
    import numpy as np
    testing = [text]
    testing= pd.Series(testing)
    testing = testing.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    # testing_encode = tokenizer.encode(testing,add_special_tokens=True)
    test_pad = np.array([i + [0]*(max_len-len(i)) for i in testing.values])
    attention_mask = np.where(test_pad != 0, 1, 0)
    input_ids = torch.tensor(test_pad)  
    attention_mask = torch.tensor(attention_mask)

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
    checking = last_hidden_states[0][:,0,:].numpy()
    return checking


def test_script(msg):
    test_msg = msg
    embedings = test_samples(model, tokenizer, test_msg, 28)
    return embedings


In [88]:
embeddings = test_script("how to claim an expense")


In [89]:
# embeddings

In [1]:
D, I = index.search(embeddings, 5)

NameError: name 'index' is not defined

In [73]:
final = []
for each in sol[1][0]:
    final.append(each)

In [74]:
final

[544, 510, 550, 531, 534]

In [75]:
import pandas as pd
df = pd.read_csv("hrdata.csv")

In [76]:
df.iloc[final]

,Text,Intent
544,What are the considerations for tracking atten...,wfh_attendance
510,What role does technology play in monitoring a...,wfh_attendance
550,How do companies handle attendance tracking fo...,wfh_attendance
531,Can you explain the role of HR in monitoring a...,wfh_attendance
534,What are the considerations for tracking atten...,wfh_attendance


In [53]:



def split_data(features,labels):
    from sklearn.model_selection import train_test_split
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels,test_size=0.3,random_state=0,stratify=labels)
    return train_features, test_features, train_labels, test_labels




def model_metrics(model, test_features, test_labels):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score,f1_score,recall_score
    out=model.predict(test_features)
    print("Accuracy : ",accuracy_score(out,test_labels)*100, flush=True)
    print("Precision : ",precision_score(out,test_labels,average = 'weighted')*100, flush=True)
    print("f1_score : ",f1_score(out,test_labels,average = "weighted")*100, flush=True)
    print("recall score : ",recall_score(out,test_labels,average="weighted")*100, flush=True)
    print("---------------------------------------------------------------------------", flush=True)


def random_forest_model(train_features, train_labels, test_features, test_labels, input_params={}, cv=5):
    from sklearn.ensemble import RandomForestClassifier
    from skopt import BayesSearchCV
    from skopt.space import Categorical,Real,Integer
    rc = RandomForestClassifier()
    if not input_params:
        params = {
        'n_estimators' : Integer(80,120),
        'max_depth' : Integer(4,8),
       
        'min_samples_split' : Integer(4,6)
        }
    else:
        params = input_params
    bayes_search = BayesSearchCV(rc,params, cv=cv)
    bayes_search.fit(train_features,train_labels)
    print("-----------------------------------------------------------------------------", flush=True)
    print("Random Forest model", flush=True)
    model_metrics(bayes_search, test_features, test_labels)
    return bayes_search




In [54]:
features = final_data_distil_bert
df = pd.read_csv("labels_generated.csv")
labels = df["intent"]
train_features, test_features, train_labels, test_labels = split_data(features, labels)


In [55]:
import pickle
pickle.dump(random_forest_model(train_features, train_labels, test_features, test_labels),open('distil_random.sav','wb'))

KeyboardInterrupt: 

In [57]:
random_model = pickle.load(open('distil_random.sav', 'rb'))

In [64]:
embeddings = test_script("how to claim an expense")
random_model.predict(embeddings)

array([9], dtype=int64)